# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846845115092                   -0.70    4.8
  2   -7.852312288761       -2.26       -1.53    1.0
  3   -7.852646043515       -3.48       -2.52    3.2
  4   -7.852646676726       -6.20       -3.38    2.0
  5   -7.852646685934       -8.04       -4.74    1.5
  6   -7.852646686724       -9.10       -5.22    3.8
  7   -7.852646686730      -11.25       -6.07    1.0
  8   -7.852646686730      -12.54       -7.64    2.5
  9   -7.852646686730   +    -Inf       -7.54    2.5
 10   -7.852646686730      -14.35       -8.55    2.0
 11   -7.852646686730   +  -14.57       -9.21    1.2
 12   -7.852646686730      -14.75       -9.79    1.8
 13   -7.852646686730   +  -14.75      -10.27    2.2
 14   -7.852646686730   +    -Inf      -10.70    1.2
 15   -7.852646686730   +  -14.75      -11.93    2.2
 16   -7.852646686730      -14.45      -12.18    2.2


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846823480195                   -0.70           4.5
  2   -7.852524714866       -2.24       -1.63   0.80    2.0
  3   -7.852636788557       -3.95       -2.70   0.80    1.0
  4   -7.852646472295       -5.01       -3.31   0.80    2.2
  5   -7.852646678461       -6.69       -4.15   0.80    1.2
  6   -7.852646686429       -8.10       -4.77   0.80    1.5
  7   -7.852646686719       -9.54       -5.61   0.80    1.8
  8   -7.852646686730      -10.99       -6.70   0.80    2.0
  9   -7.852646686730      -12.55       -7.43   0.80    2.8
 10   -7.852646686730      -15.05       -8.28   0.80    1.2
 11   -7.852646686730      -15.05       -9.18   0.80    2.2
 12   -7.852646686730   +  -14.75       -9.96   0.80    2.0
 13   -7.852646686730   +    -Inf      -10.95   0.80    2.0
 14   -7.852646686730      -14.57      -11.01   0.80    2.5
 15   -7.852646686730   +  -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.357026e+01     2.881777e+00
 * time: 0.40321993827819824
     1     1.317197e+00     1.649172e+00
 * time: 0.6071429252624512
     2    -1.525467e+00     1.991861e+00
 * time: 0.6302111148834229
     3    -3.870550e+00     1.626408e+00
 * time: 0.6633601188659668
     4    -5.160411e+00     1.488371e+00
 * time: 0.6964669227600098
     5    -6.886063e+00     1.058968e+00
 * time: 0.7297520637512207
     6    -7.513658e+00     3.957578e-01
 * time: 0.7632081508636475
     7    -7.705365e+00     1.479742e-01
 * time: 0.7865550518035889
     8    -7.786868e+00     1.354389e-01
 * time: 0.8099009990692139
     9    -7.825310e+00     6.541721e-02
 * time: 0.8330819606781006
    10    -7.841041e+00     6.471218e-02
 * time: 0.8561999797821045
    11    -7.845943e+00     5.512835e-02
 * time: 0.8792519569396973
    12    -7.847689e+00     3.706264e-02
 * time: 0.9022951126098633
    13    -7.849728e+00     2.908303e-02
 * time: 0.925475120

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846794984898                   -0.70    4.8
  2   -7.852285879979       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646685659                   -2.54
  2   -7.852646686730       -8.97       -5.43
  3   -7.852646686730   +  -14.75      -10.07
  4   -7.852646686730   +    -Inf      -15.56


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.4772429603418834e-13
|ρ_newton - ρ_scfv| = 1.66183683616696e-13
|ρ_newton - ρ_dm|   = 1.6094878185504857e-9
